In [19]:
import os
import requests
import pandas as pd
import spotipy
import spotipy.util as util
import json
from spotipy.oauth2 import SpotifyClientCredentials
from pprint import pprint
from time import sleep

In [26]:
# Spotify API Keys
import config
os.environ['SPOTIPY_CLIENT_ID']=config.spotify_cliend_id
os.environ['SPOTIPY_CLIENT_SECRET']=config.spotify_client_secret

In [27]:
# Generate API Token
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
token = client_credentials_manager.get_access_token()
# print(token)

In [31]:
# Load clean combined weekly data
FORGOTIFY_SONG_FILE_NAME = os.path.join('data', 'us', 'forgotify-10.csv')
forgotify_song_df = pd.read_csv(FORGOTIFY_SONG_FILE_NAME)
forgotify_song_df.head()


,Position,Artist,Title
0,1,Alex Kiskachi,On Land and Sea
1,2,Ragged Jack,What We've Got Coming
2,3,Kidzone,Dem Bones
3,4,Armen Firman,Dressed For War
4,5,Dave Hancock,It Might As Well Be Spring


In [32]:
features_list = []
for position in forgotify_song_df['Position']:
    title = forgotify_song_df.loc[forgotify_song_df['Position']==position, 'Title'].item()
    artist = forgotify_song_df.loc[forgotify_song_df['Position']==position, 'Artist'].item()
    results = sp.search(q=f'track:{title} artist:{artist}', type='track', limit=1)
    track_id = results['tracks']['items'][0]['id']
    print(f'{position} - {track_id}')
    feature_url = 'https://api.spotify.com/v1/audio-features/{0}'.format(track_id)
    feature_rsp = requests.get(feature_url, headers={'Authorization': 'Bearer '+token})
    features = feature_rsp.json()
    
    track_rsp = requests.get(features['track_href'], headers={'Authorization': 'Bearer '+token})
    track_info = track_rsp.json()
    
    features_as_dict = {
        'Position':position,
        'URL': track_info['external_urls']['spotify'],
        'Acousticness': features['acousticness'],
        'Danceability': features['danceability'],
        'Duration (ms)': features['duration_ms'],
        'Energy': features['energy'],
        'Instrumentalness': features['instrumentalness'],
        'Key': features['key'],
        'Liveness': features['liveness'],
        'Loudness': features['loudness'],
        'Mode': features['mode'],
        'Speechiness': features['speechiness'],
        'Tempo': features['tempo'],
        'Time Signature': features['time_signature'],
        'Valence': features['valence'],
        'Album': track_info['album']['name'],
        'Image': track_info['album']['images'][1]['url'],
        'Explicit': track_info['explicit'],
        'Popularity': track_info['popularity']
    }
    features_list.append(features_as_dict)
    sleep(1) # Throttle API calls
 
features_df = pd.DataFrame(features_list)
enriched_df = pd.merge(forgotify_song_df, features_df, how='left', on='Position')
enriched_df.head()

1 - 0R81dN7CIDAoRqWjkPflD7
2 - 1HRbMCrcBf0yWdHsfBIjrf
3 - 0XeuKYlH6mLZR24gJgjjE7
4 - 1Mo1SNRohQlkpMdpX7womI
5 - 5UqHkqWbTKhGhuRDZJXeX8
6 - 7xbDPQuuX2aayXZno6J6ai
7 - 2XASXobIEDCvhKHkjiEb3m
8 - 3O2d7OIK4FTANICPTvpcvv
9 - 4Hl9z3oGffz2FazVdDZE82
10 - 4HVzOc96oaKcxeDlKPBGmF


,Position,Artist,Title,Acousticness,Album,Danceability,Duration (ms),Energy,Explicit,Image,...,Key,Liveness,Loudness,Mode,Popularity,Speechiness,Tempo,Time Signature,URL,Valence
0,1,Alex Kiskachi,On Land and Sea,0.740,Unfaithful Syncopations,0.747,260667,0.2060,False,https://i.scdn.co/image/cd2e8a609beda057519e55...,...,4,0.1180,-17.181,0,0,0.0528,126.951,4,https://open.spotify.com/track/0R81dN7CIDAoRqW...,0.436
1,2,Ragged Jack,What We've Got Coming,0.133,Life Is A Wheel,0.568,287333,0.4430,False,https://i.scdn.co/image/c5d434a0a6177ca7ee4d75...,...,0,0.0764,-10.865,1,0,0.0295,140.074,4,https://open.spotify.com/track/1HRbMCrcBf0yWdH...,0.141
2,3,Kidzone,Dem Bones,0.163,Songs for Having Fun With My Friends,0.820,69147,0.3820,False,https://i.scdn.co/image/bb81d6c718aefe8ac93fe1...,...,2,0.0770,-11.985,1,0,0.1410,161.597,4,https://open.spotify.com/track/0XeuKYlH6mLZR24...,0.853
3,4,Armen Firman,Dressed For War,0.228,Dressed For War,0.488,265476,0.5700,False,https://i.scdn.co/image/e44af6bc97a5712ada2a50...,...,0,0.0896,-5.702,0,0,0.0328,164.031,4,https://open.spotify.com/track/1Mo1SNRohQlkpMd...,0.226
4,5,Dave Hancock,It Might As Well Be Spring,0.962,Out of Nowhere,0.402,359360,0.0367,False,https://i.scdn.co/image/2194a828debdbf36cc8407...,...,7,0.0776,-16.049,1,0,0.0597,75.414,3,https://open.spotify.com/track/5UqHkqWbTKhGhuR...,0.229


In [37]:
ENRICHED_FILE_NAME = os.path.join('data', 'clean', 'forgotify-10-enriched.csv')

if not os.path.exists(os.path.dirname(ENRICHED_FILE_NAME)):
    os.mkdir(os.path.dirname(ENRICHED_FILE_NAME))
    
enriched_df.to_csv(ENRICHED_FILE_NAME)